In [1]:
import pandas as pd
import math
import string
import re
from myFunctions import processSentences, meanVector, cosineDistance

import statistics



# NLP Packages
from gensim.models.phrases import Phrases, Phraser
import gensim.downloader as api
from gensim.models import Word2Vec

/Users/brianward/Desktop/Northeastern/AI/project/aiEnv/lib/python3.9/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


# data import

Okay so Kaggle has two pretty awesome wallstreet bets datasets.  
One only has the title but has a LOT of posts,  
and the other has a body and titles.  
Im going to only use the body from the second source and use all the titles from the other. 

In [2]:
reddit = pd.read_csv("data/reddit_wsb.csv")
reddit2 = pd.read_csv("data/r_wallstreetbets_posts.csv")

/Users/brianward/Desktop/Northeastern/AI/project/aiEnv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,7) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
print('length reddit :', len(reddit))
print('legnth reddit without NA :', len(reddit.dropna()))

reddit.head(1)
# SO what do we actually want to train the model on?
# we can use the titles and we can use the bodys. 
# we also dont really need to store any of this other information for model training.



length reddit : 43057
legnth reddit without NA : 20678


,title,score,id,url,comms_num,created,body,timestamp
0,"It's not about the money, it's about sending a...",55,l6ulcx,https://v.redd.it/6j75regs72e61,6,1.611863e+09,NaN,2021-01-28 21:37:41


In [4]:
# Should we parse each body into sentences split on '.' probably...
# okay so lets go ahead and do this first, we can also treat each title as a its own sentence.

sentences_raw = []


for i in range(len(reddit)):
    body = reddit['body'][i]
    if isinstance(body, str):
        sentences_raw.extend(body.split('.'))
    

In [5]:
# how many sentences do we have now?

len(sentences_raw)

280855

# second datasource

In [6]:
len(reddit2)+len(reddit)

1161920

In [7]:
# okay so now were going to add these titles to our sentences.

for i in range(len(reddit2)):
    title = reddit2['title'][i]
    if isinstance(title, str):
        sentences_raw.append(title)

In [8]:
print('length of raw sentences : ', len(sentences_raw))

length of raw sentences :  1399717


# Pre-Processing

Okay so now we need to process these strings to prep them for the model training.   
What we will do :  
* string cleaning
* tokenizing sentences
* creating bigrams
* Should we remove stopwords?



# String Cleaning 
* lowercase 
* remove punctuation
* remove numbers 
* clean whitespace
* tokenize
* find bigrams
* find trigrams

In [9]:
# apply our string cleaning. 
sentences_clean = processSentences(sentences_raw, minStringSize = 5, minTokenCount = 3, splitonPeriod = True, phraseMinCount = 25, bigrams=True, trigrams=False)



In [10]:
# now lets take a look at where we're at:
sentences_clean[0]

['the',
 'ceo',
 'of',
 'nasdaq',
 'pushed',
 'to',
 'halt_trading',
 '“to',
 'give',
 'investors',
 'a',
 'chance',
 'to',
 'recalibrate',
 'their',
 'positions”']

In [11]:
print('so we have {} sentences to train our model on'.format(len(sentences_clean)))
# damn thats alot hell yea.

so we have 1205382 sentences to train our model on


In [12]:
sentences_clean[501]

['“these',
 'small',
 'and',
 'unsophisticated',
 'investors',
 'are',
 'probably',
 'going',
 'to',
 'get',
 'hurt',
 'by',
 'this']

# Import Text8

In [13]:
text8 = api.load('text8')

# Model Training

In [14]:
# start with text8 as a base. 
model = Word2Vec(text8, min_count=25, window =2, sg = 1, vector_size=100)

In [15]:
# we can then continue to train on our model on the sentences we have created.
model.build_vocab(sentences_clean, update=True)
model.train(sentences_clean,total_examples=len(sentences_clean), epochs = 10)


(98296687, 126667930)

In [16]:
model.wv.most_similar(positive=['gme'])

[('amc', 0.8408816456794739),
 ('pltr', 0.8126963973045349),
 ('bb', 0.8063815832138062),
 ('gamestop', 0.7710800766944885),
 ('nok', 0.7422802448272705),
 ('gma', 0.7160327434539795),
 ('gamestonk', 0.7137332558631897),
 ('game_stop', 0.7121042609214783),
 ('nakd', 0.6815323829650879),
 ('gmc', 0.6784548759460449)]

In [17]:
# we dont need to save the whole model (only for further training) instead we will save the wordvectors

# model.wv.save('V2.wordvectors')

In [18]:
# quick test:

# another way to test this is to do it by set agian...

def testV2(keyedVectors, myVec, groundTruth, threshold):
    
    vocab = set()
    expected = set()
    
    for i in range(len(groundTruth)):
        
        for word in groundTruth['sent'][i].split(' '):
            if word in keyedVectors:
                vocab.add(word)
        
        for word2 in groundTruth['expected'][i]:
            if word2 in keyedVectors:
                expected.add(word2)

    
    # okay so calculate missed:    
    missed = set()
    
    for word in expected:
        if cosineDistance(myVec, keyedVectors[word]) < threshold :
            missed.add(word)
    
    # first need to subtract the expected set.
    notExpected = vocab - expected
    
    over = set()
    
    for word in notExpected:
        if cosineDistance(myVec, keyedVectors[word]) >= threshold :
            over.add(word)
    missedScore = (len(expected)-len(missed))/len(expected)
    overScore = (len(notExpected)-len(over))/len(notExpected)
    
    return [missedScore, overScore, statistics.mean([missedScore, overScore])]

In [19]:
groundTruth = pd.read_csv("data/groundTruth.csv")
groundTruth = groundTruth[['sent','expected']]

def tokenize(sent):
    if type(sent) == str:
        return sent.strip().split(' ')
    else: 
        return []

    
# apply to our expected column
groundTruth['expected'] = groundTruth['expected'].apply(tokenize)

In [20]:
myVec = meanVector(model.wv, positive=['gme', 'amc','game_stop','tsla', 'nok', 'etsy', 'crox', 'appl', 'microsoft', 'facebook', 'netflix', 'dropbox','slack', 'peloton', 'wal_mart', 'lululemon'], 
                negative=[])

In [21]:
testV2(model.wv, myVec, groundTruth, 0.55)

[0.6842105263157895, 0.979890310786106, 0.8320504185509477]